<a href="https://colab.research.google.com/github/TomAmster/Data-Science/blob/master/2_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Overview & Introduction

1.   Loading and Downloading Datasets
2.   Formatting ICD Codes
3.   Merging Tokenized Note Summaries with ICD Code Labels Based on Admission IDs
4.   Splitting The Merged Data Set to Train, Dev and Test Datasets.
5. Building Word Vocabulary Base On Train Dataset
6. Filtering Top 50 ICD Codes Notes



In [0]:
import pandas as pd
import itertools
from google.colab import drive
import matplotlib.pyplot as plt
import csv
import seaborn as sns 
import numpy as np
from  numpy import int32
from collections import Counter, defaultdict
from os import path

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
!pip install jsonlines
import jsonlines

In [0]:
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 1.Preparing The Dataset


## 1.1 Donwload the Dataset
*   Download the MIMIC Dataset from Physionet
*   Unzipthe relevent schemas
*   Move the Files to Google Drive

In [0]:
# !wget -r -N -c -np --user tomamsterdam --ask-password https://physionet.org/files/mimiciii/1.4/
# !gunzip  '/content/physionet.org/files/mimiciii/1.4/DIAGNOSES_ICD.csv.gz'
#!mv '/content/physionet.org/files/mimiciii/1.4/DIAGNOSES_ICD.csv' '/content/drive/Shared drives/NLP Seminar/Data'

## 1.2 Load the Note Event Dataset
This dataset includes the note summaries for the hospital hadmission.
Columns are as follows:


*   Subject ID - Unqiue ID for patient in the hospital
*   HADM_ID   -  Unqiue ID for an hospital admission
*   Chart Date - Time stamp
*   Category - The type of the events summary
*   Text - Long document of doctor notes



In [0]:
dfproc = pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/NOTEEVENTS.csv')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
dfproc.shape

(2083180, 11)

## 1.3 Split The Dataset To Smaller Chunks
Due to the dataset size, we split the dataset to smaller chunks to be able to work with limited RAM resources


In [0]:
df1 = dfproc.iloc[:500000, :]
df2 = dfproc.iloc[500000:1000000, :]
df3 = dfproc.iloc[1000000:1500000, :]
df4 = dfproc.iloc[1500000:,:]

In [0]:
print(df1.shape)
df1.head()

(500000, 11)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [0]:
print(df2.shape)
df2.head()

(500000, 11)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
500000,505332,55514,170308.0,2193-01-04,2193-01-04 18:50:00,2193-01-04 18:50:29,Respiratory,Respiratory Care Shift Note,14913.0,NaN,TITLE:\n Demographics\n Day of intubation:...
500001,505399,67188,108429.0,2115-12-28,2115-12-28 06:17:00,2115-12-28 06:17:22,Nursing,Nursing Progress Note,21476.0,NaN,"Pain control (acute pain, chronic pain)\n As..."
500002,505400,54461,110002.0,2117-11-13,2117-11-13 01:30:00,2117-11-13 06:17:53,Nursing,Nursing Progress Note,16037.0,NaN,TITLE: The patient is a 37y/o male with a PMH ...
500003,505401,62143,167116.0,2102-12-16,2102-12-16 02:47:00,2102-12-16 06:23:08,Nursing,Nursing Progress Note,16302.0,NaN,67M with ARDS thought secondary to H1N1; cours...
500004,505497,82354,199749.0,2196-12-17,2196-12-17 11:45:00,2196-12-17 12:14:19,Nursing,Nursing Transfer Note,20818.0,NaN,"73 yo gentleman with PMH significant for CVA, ..."


In [0]:
print(df3.shape)
df3.head()

(500000, 11)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
1000000,1004752,27190,118590.0,2192-04-28,2192-04-28 11:26:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2192-4-28**] 11:26 AM\n CHEST (PORTABLE AP)...
1000001,1004753,28481,189424.0,2112-02-27,2112-02-27 11:27:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2112-2-27**] 11:27 AM\n CHEST (PORTABLE AP)...
1000002,1002969,31123,154148.0,2114-02-13,2114-02-13 20:32:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2114-2-13**] 8:32 PM\n CHEST (PORTABLE AP);...
1000003,1003006,711,158767.0,2185-04-06,2185-04-06 06:11:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2185-4-6**] 6:11 AM\n CHEST (PORTABLE AP) ...
1000004,1003621,31301,169336.0,2138-02-15,2138-02-15 18:05:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2138-2-15**] 6:05 PM\n CHEST (PORTABLE AP) ...


In [0]:
print(df4.shape)
df4.head()

(583180, 11)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
1500000,1527953,20190,145260.0,2138-08-20,2138-08-20 17:31:00,2138-08-20 17:41:00,Nursing/other,Report,17582.0,NaN,"7A-7P UPDATE\nCV: HR ST, NO ECTOPY, SBP STABLE..."
1500001,1522714,19815,154936.0,2137-10-24,2137-10-24 04:38:00,2137-10-24 04:52:00,Nursing/other,Report,15830.0,NaN,Nursing Process Note: 1900-0700\nSignificant e...
1500002,1522715,19815,154936.0,2137-10-24,2137-10-24 06:14:00,2137-10-24 06:17:00,Nursing/other,Report,20955.0,NaN,RESPIRATORY CARE NOTE\n\nPT REMAINS [**Name (N...
1500003,1527611,20181,120654.0,2173-11-09,2173-11-09 06:43:00,2173-11-09 06:46:00,Nursing/other,Report,18725.0,NaN,RESP CARE: Pt remains trache [**Male First Nam...
1500004,1527612,20181,120654.0,2173-11-09,2173-11-09 17:39:00,2173-11-09 17:46:00,Nursing/other,Report,15627.0,NaN,nursing note: 7a-7p\nneuro- pt awake and [** *...


## 1.4 Save Splitted Datasets

In [0]:
df1.to_csv('/content/drive/Shared drives/NLP Seminar/Data/NOTEEVENTS_SPLIT_1.csv')
df2.to_csv('/content/drive/Shared drives/NLP Seminar/Data/NOTEEVENTS_SPLIT_2.csv')
df3.to_csv('/content/drive/Shared drives/NLP Seminar/Data/NOTEEVENTS_SPLIT_3.csv')
df4.to_csv('/content/drive/Shared drives/NLP Seminar/Data/NOTEEVENTS_SPLIT_4.csv')

# 2.Correlate The Note Summaries With ICD Labels

## 2.1 Load Diagnosis and Procedure ICD Codes

The codes in MIMIC-III are given in separate files for procedures and diagnoses

In [0]:
dfproc = pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/PROCEDURES_ICD.csv', dtype={'SUBJECT_ID':int32, 'HADM_ID':int32, 'ICD9_CODE':str}, keep_default_na=False, na_values='')
dfproc_desc = pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/D_ICD_PROCEDURES.csv', dtype={'ICD9_CODE':str}, keep_default_na=False, na_values='')
dfdiag = pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/DIAGNOSES_ICD.csv', dtype={'SUBJECT_ID':int32, 'HADM_ID':int32, 'ICD9_CODE':str}, keep_default_na=False, na_values='')
dfdiag_desc = pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/D_ICD_DIAGNOSES.csv', dtype={'ICD9_CODE':str}, keep_default_na=False, na_values='')
dfproc['ICD9_CODE'] = dfproc['ICD9_CODE'].fillna('')
dfproc = dfproc.dropna(axis=0, subset=['SUBJECT_ID', 'HADM_ID'])
dfdiag['ICD9_CODE'] = dfdiag['ICD9_CODE'].fillna('')
dfdiag = dfdiag.dropna(axis=0, subset=['SUBJECT_ID', 'HADM_ID'])

## 2.2 Reformat ICD Code Labels
The codes are given without periods, which might lead to collisions if we naively combine them. So we have to add the periods back in the right place.

In [0]:
dfdiag['ICD9_CODE'] = dfdiag.apply(lambda row: str(reformat(str(row[4]), True)), axis=1)
dfdiag_desc['ICD9_CODE'] = dfdiag.apply(lambda row: str(reformat(str(row[4]), True)), axis=1)
dfproc['ICD9_CODE'] = dfproc.apply(lambda row: str(reformat(str(row[4]), False)), axis=1)
dfproc_desc['ICD9_CODE'] = dfproc.apply(lambda row: str(reformat(str(row[4]), False)), axis=1)
dfcodes = pd.concat([dfdiag, dfproc])

In [0]:
print(dfcodes.shape)
dfcodes.head()

## 2.2 Load Pre Tokenized Note Summaries
*   Select only discharge summaries and their addenda

In [0]:
df = pd.read_json('/content/drive/Shared drives/NLP Seminar/Data/NOTEEVENTS_SPLIT_1.csvnotes_tokenized.ndjson', lines=True, dtype={0:int32, 1:int32, 5:bool, 6:int32})
df.columns = ['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CATEGORY', 'DESCRIPTION', 'ISERROR', 'N_TOKENS', 'TEXT']
df = df[(df['CATEGORY'] == 'Discharge summary') & (df['ISERROR'] == False)]

In [0]:
print(df.shape)
df.head()

(59652, 8)


,SUBJECT_ID,HADM_ID,CHARTDATE,CATEGORY,DESCRIPTION,ISERROR,N_TOKENS,TEXT
0,22532,167853,2151-08-04,Discharge summary,Report,False,97,"[[admission, date], [july], [discharge, date],..."
1,13702,107527,2118-06-14,Discharge summary,Report,False,1803,"[[admission, date], [june], [discharge, date],..."
2,13702,167118,2119-05-25,Discharge summary,Report,False,1345,"[[admission, date], [may], [discharge, date], ..."
3,13702,196489,2124-08-18,Discharge summary,Report,False,2375,"[[admission, date], [july], [discharge, date],..."
4,26880,135453,2162-03-25,Discharge summary,Report,False,2012,"[[admission, date], [march], [discharge, date]..."


In [0]:
df = df.sort_values(['CHARTDATE']).groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).agg({'TEXT': lambda doclist : list(itertools.chain(sent for doc in doclist for sent in doc)),'N_TOKENS': sum})

In [0]:
df.head()

,SUBJECT_ID,HADM_ID,TEXT,N_TOKENS
0,3,145834,"[[admission, date], [october], [discharge, dat...",2112
1,4,185777,"[[admission, date], [march], [discharge, date]...",1429
2,6,107064,"[[admission, date], [may], [discharge, date], ...",1026
3,9,150750,"[[admission, date], [november], [discharge, da...",1369
4,10,184167,"[[admission, date], [june], [discharge, date],...",507


In [0]:
distinct_tok = len(set(token for doc in df['TEXT'] for sentence in doc for token in sentence))
num_tok = df['N_TOKENS'].sum()

In [0]:
print("Num distinct tokens", str(distinct_tok))
print("Num tokens", str(num_tok))

Num distinct tokens 154552
Num tokens 79335746


In [0]:
len(df['HADM_ID'].unique()), len(dfcodes['HADM_ID'].unique())

(52726, 58976)

## 2.3 Merge Tokenized Notes With Labels Set
There were some HADM_ID's that didn't have discharge summaries, so they weren't included with our notes

In [0]:
hadm_ids = pd.DataFrame(df['HADM_ID'].unique(), columns=['HADM_ID'], dtype=int32)
dfcodes = pd.merge(dfcodes, hadm_ids, how='inner', on=['HADM_ID'])

In [0]:
len(dfcodes['HADM_ID'].unique()), len(dfcodes['ICD9_CODE'].unique())

(52726, 8930)

In [0]:
dfcodes_grouped = dfcodes.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False).agg({'ICD9_CODE' : lambda codes: list([str(code) for code in codes if len(str(code)) > 0])}).rename(columns={'ICD9_CODE':'LABELS'})
dflabeled = pd.merge(df, dfcodes_grouped, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
with jsonlines.open("/content/drive/Shared drives/NLP Seminar/Data/notes_1_labeled.ndjson", 'w') as labeled:
    for index, note in dflabeled.iterrows():
        labeled.write(note.tolist())

In [0]:
dflabeled.to_csv("/content/drive/Shared drives/NLP Seminar/Data/NOTEEVENTS_SPLIT_1_TOKENIZED_LABELED.csv")

In [0]:
cnt = Counter([token for doc in dflabeled['TEXT'] for sentence in doc for token in sentence])
distinct_tok = len(cnt.keys())
num_tok = sum(cnt.values())


In [0]:
print("num distinct tokens", len(cnt.keys()), "num tokens", sum(cnt.values()))

In [0]:
len(dflabeled['HADM_ID'].unique())

In [0]:
dflabeled.shape

In [0]:
dflabeled.head(3)

# 3.Create Train, Dev & Test Dataset

In [0]:
dflabeled = pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/NOTEEVENTS_SPLIT_1_TOKENIZED_LABELED.csv')

## 3.1 Split Admission-Ids to Train, Dev & Test With Ratio - 70%/20%/10%

In [0]:
all_hadm_ids = dflabeled['HADM_ID']
print(all_hadm_ids.shape)
print(all_hadm_ids.head(3))

(52726,)
0    145834
1    185777
2    107064
Name: HADM_ID, dtype: int64


In [0]:
train_hadm_ids = all_hadm_ids[ : 36908]
dev_hadm_ids = all_hadm_ids[36908 :47454 ]
test_hadm_ids = all_hadm_ids[47454 : ]

In [0]:
print(train_hadm_ids.shape)
print(train_hadm_ids.head(3))
print(dev_hadm_ids.shape)
print(dev_hadm_ids.head(3))
print(test_hadm_ids.shape)
print(test_hadm_ids.head(3))

(36908,)
0    145834
1    185777
2    107064
Name: HADM_ID, dtype: int64
(10546,)
36908    153312
36909    185833
36910    183006
Name: HADM_ID, dtype: int64
(5272,)
47454    177315
47455    192175
47456    112121
Name: HADM_ID, dtype: int64


In [0]:
train_hadm_ids=train_hadm_ids.reset_index()
dev_hadm_ids=dev_hadm_ids.reset_index()
test_hadm_ids=test_hadm_ids.reset_index()


In [0]:
#dev_hadm_ids=dev_hadm_ids.drop('index',1)
#test_hadm_ids=test_hadm_ids.drop('index',1)
#train_hadm_ids=train_hadm_ids.drop('index',1)
train_hadm_ids.set_index('HADM_ID', inplace=True)
dev_hadm_ids.set_index('HADM_ID', inplace=True)
test_hadm_ids.set_index('HADM_ID', inplace=True)

## 3.2 Save Admission Ids Keys For Future Dataset Split

In [0]:
dev_hadm_ids.to_csv('/content/drive/Shared drives/NLP Seminar/Data/dev_hadm_ids.csv')
test_hadm_ids.to_csv('/content/drive/Shared drives/NLP Seminar/Data/test_hadm_ids.csv')
train_hadm_ids.to_csv('/content/drive/Shared drives/NLP Seminar/Data/train_hadm_ids.csv')

In [0]:
dev_hadm_ids= pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/dev_hadm_ids.csv')
test_hadm_ids  = pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/test_hadm_ids.csv')
train_hadm_id = pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/train_hadm_ids.csv')

## 3.3 Split Dataset to 3 Parts

In [0]:
base_name = '/content/drive/Shared drives/NLP Seminar/Data/notes_labeled'
SPLITS_DIR='/content/drive/Shared drives/NLP Seminar/Data/'
splits = {}
for split in ['train', 'dev', 'test']:
    split_ids_df = pd.read_csv(path.join(SPLITS_DIR, '{}_hadm_ids.csv'.format(split)), names=['HADM_ID'], dtype=int32)
    splits[split] = pd.merge(dflabeled, split_ids_df, how='inner', on=['HADM_ID'])
    with jsonlines.open('{}_{}.ndjson'.format(base_name, split), 'w') as split_out:
        for index, row in splits[split].iterrows():
           split_out.write(row.tolist())

In [0]:
print(splits)

{'train':        Unnamed: 0  ...                                             LABELS
0               0  ...  ['038.9', '785.59', '584.9', '427.5', '410.71'...
1               1  ...  ['042', '136.3', '799.4', '276.3', '790.7', '5...
2               2  ...  ['403.91', '444.0', '997.2', '276.6', '276.7',...
3               3  ...  ['431', '507.0', '428.0', '584.9', '276.5', '4...
4               4  ...  ['V30.00', '774.2', '765.25', '765.15', 'V29.0...
...           ...  ...                                                ...
36903       36903  ...  ['038.9', '507.0', '584.9', '785.52', '995.92'...
36904       36904  ...  ['414.01', '585.6', '411.1', '458.21', '427.31...
36905       36905  ...  ['682.3', '428.23', '585.6', '584.9', '530.20'...
36906       36906  ...  ['038.9', '569.83', '785.52', '486', '427.1', ...
36907       36907  ...  ['153.6', '507.0', '584.9', '511.9', '427.32',...

[36908 rows x 6 columns], 'dev':        Unnamed: 0  ...                                             L

## 3.4 Clean and Save Train/Dev/Test datasets as CSV and JSON

In [0]:
notes_labeled_train = splits['train']
notes_labeled_test = splits['dev']
notes_labeled_dev = splits['test']

In [0]:
notes_labeled_train.head()

,Unnamed: 0,SUBJECT_ID,HADM_ID,TEXT,N_TOKENS,LABELS
0,0,3,145834,"[['admission', 'date'], ['october'], ['dischar...",2112,"['038.9', '785.59', '584.9', '427.5', '410.71'..."
1,1,4,185777,"[['admission', 'date'], ['march'], ['discharge...",1429,"['042', '136.3', '799.4', '276.3', '790.7', '5..."
2,2,6,107064,"[['admission', 'date'], ['may'], ['discharge',...",1026,"['403.91', '444.0', '997.2', '276.6', '276.7',..."
3,3,9,150750,"[['admission', 'date'], ['november'], ['discha...",1369,"['431', '507.0', '428.0', '584.9', '276.5', '4..."
4,4,10,184167,"[['admission', 'date'], ['june'], ['discharge'...",507,"['V30.00', '774.2', '765.25', '765.15', 'V29.0..."


In [0]:
notes_labeled_train=notes_labeled_train.drop('SUBJECT_ID',1)
notes_labeled_train=notes_labeled_train.drop('Unnamed: 0',1)
notes_labeled_dev=notes_labeled_dev.drop('SUBJECT_ID',1)
notes_labeled_dev=notes_labeled_dev.drop('Unnamed: 0',1)
notes_labeled_test=notes_labeled_test.drop('SUBJECT_ID',1)
notes_labeled_test=notes_labeled_test.drop('Unnamed: 0',1)

In [0]:
splits['train'].head()
splits['train']=splits['train'].drop('SUBJECT_ID',1)
splits['train']=splits['train'].drop('Unnamed: 0',1)
splits['dev']= splits['dev'].drop('SUBJECT_ID',1)
splits['dev'] = splits['dev'].drop('Unnamed: 0',1)
splits['test']=splits['test'].drop('SUBJECT_ID',1)
splits['test']=splits['test'].drop('Unnamed: 0',1)

In [0]:
base_name = '/content/drive/Shared drives/NLP Seminar/Data/notes_labeled'
for split in ['train', 'dev', 'test']:
    with jsonlines.open('{}_{}.ndjson'.format(base_name, split), 'w') as split_out:
        for index, row in splits[split].iterrows():
           split_out.write(row.tolist())

In [0]:
notes_labeled_train.to_csv('/content/drive/Shared drives/NLP Seminar/Data/notes_labeled_train', index=False)
notes_labeled_test.to_csv('/content/drive/Shared drives/NLP Seminar/Data/notes_labeled_test', index=False)
notes_labeled_dev.to_csv('/content/drive/Shared drives/NLP Seminar/Data/notes_labeled_dev', index=False)

# 4.Build Words Vocabulary

## 4.1 Load Train Dataset

In [0]:
notes_labeled_train = pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/notes_labeled_train')

In [0]:
notes_labeled_train.head(3)

,HADM_ID,TEXT,N_TOKENS,LABELS
0,145834,"[['admission', 'date'], ['october'], ['dischar...",2112,"['038.9', '785.59', '584.9', '427.5', '410.71'..."
1,185777,"[['admission', 'date'], ['march'], ['discharge...",1429,"['042', '136.3', '799.4', '276.3', '790.7', '5..."
2,107064,"[['admission', 'date'], ['may'], ['discharge',...",1026,"['403.91', '444.0', '997.2', '276.6', '276.7',..."


## 4.2 Create Vocabulary Table
*   Words In Note Summaries
*   Number of time each word appeared
*   Number of document each work appeared in





In [0]:
vocab_min = 3 #discard tokens appearing in fewer than this many documents
word_cnt = Counter()
doc_cnt = Counter()
for doc in splits['train']['TEXT']:
    tokens = [token for sentence in doc for token in sentence]
    doc_cnt.update(set(tokens))
    word_cnt.update(tokens)

merged_cnt = {word:(count,doc_cnt[word]) for word, count in word_cnt.items()}
words, counts = zip(*merged_cnt.items())
word_count, doc_count = zip(*counts)

words = pd.Series(words)
word_count = pd.Series(word_count)
doc_count = pd.Series(doc_count)

vocab = pd.concat([words, doc_count, word_count], axis=1)
vocab.columns=['word', 'doc_count', 'word_count']
vocab = vocab.sort_values(by=['doc_count'], ascending=False)
vocab.to_csv('/content/drive/Shared drives/NLP Seminar/Data/word_count.csv', header=False, index=False)
vocab = vocab[vocab['doc_count'] >= vocab_min]

In [0]:
print(vocab.shape)
vocab.head()

(44075, 3)


,word,doc_count,word_count
4,of,36908,1026447
1,date,36908,147729
21,to,36893,1005498
0,admission,36890,153208
24,and,36883,1196370


## 4.3 Save Vocabulary
*   Words Only
*   Words & Counts



In [0]:
vocab.to_csv('/content/drive/Shared drives/NLP Seminar/Data/vocab.csv')

In [0]:
vocab=pd.read_csv('/content/drive/Shared drives/NLP Seminar/Data/vocab.csv')

In [0]:
with open('/content/drive/Shared drives/NLP Seminar/Data/vocab_words.csv', 'w') as f:
    f.write(pd.Series(vocab['word']).str.cat(sep='\n'))

# 5.Create Top 50 ICD Codes Dataset


## 5.1 Filter Top 50 ICD Codes By Frequency

In [0]:
K = 50
codes_top_K = dflabeled['LABELS'].apply(pd.Series).stack().reset_index(drop=True)
codes_top_K = codes_top_K.groupby(codes_top_K).count().sort_values(ascending=False).head(K)

In [0]:
codes_top_K.head(10)

401.9     20053
38.93     14444
428.0     12842
427.31    12594
414.01    12179
96.04      9932
96.6       9161
584.9      8907
250.00     8784
96.71      8619
dtype: int64

In [0]:
codes_top_K.to_csv('/content/drive/Shared drives/NLP Seminar/Data/top_{}_codes.csv'.format(K), header = False)


## 5.2 Find Admissions with Top 50 Label

In [0]:
train_df= splits['train']
train_df['TOP_K_LABEL']=0
test_df= splits['test']
test_df['TOP_K_LABEL']=0
dev_df= splits['dev']
dev_df['TOP_K_LABEL']=0

In [0]:
for index, row in train_df.iterrows():
  intersect = [ i for i in row['LABELS'] if i in codes_top_K]
  if len(intersect)>0:
    train_df.at[index,'TOP_K_LABEL'] =1
  
for index, row in dev_df.iterrows():
  intersect = [ i for i in row['LABELS'] if i in codes_top_K]
  if len(intersect)>0:
    dev_df.at[index,'TOP_K_LABEL'] =1
    
for index, row in test_df.iterrows():
  intersect = [ i for i in row['LABELS'] if i in codes_top_K]
  if len(intersect)>0:
    test_df.at[index,'TOP_K_LABEL'] =1


In [0]:
print(train_df.TOP_K_LABEL.value_counts())
print(dev_df.TOP_K_LABEL.value_counts())
print(test_df.TOP_K_LABEL.value_counts())

1    34461
0     2447
Name: TOP_K_LABEL, dtype: int64
1    10079
0      467
Name: TOP_K_LABEL, dtype: int64
1    5077
0     195
Name: TOP_K_LABEL, dtype: int64


In [0]:
train_hadm_ids_top_50 = train_df[train_df['TOP_K_LABEL']==1]
dev_hadm_ids_top_50 = dev_df[dev_df['TOP_K_LABEL']==1]
test_hadm_ids_top_50 = test_df[test_df['TOP_K_LABEL']==1]

In [0]:
train_hadm_ids_top_50=train_hadm_ids_top_50['HADM_ID']
dev_hadm_ids_top_50=dev_hadm_ids_top_50['HADM_ID']
test_hadm_ids_top_50=test_hadm_ids_top_50['HADM_ID']


## 5.3 Save Admission IDs Of Note Related To Top 50 ICD Code

In [0]:
train_hadm_ids_top_50.to_csv('/content/drive/Shared drives/NLP Seminar/Data/train_hadm_ids_top_50.csv',header=False, index=False)
dev_hadm_ids_top_50.to_csv('/content/drive/Shared drives/NLP Seminar/Data/dev_hadm_ids_top_50.csv',header=False, index=False)
test_hadm_ids_top_50.to_csv('/content/drive/Shared drives/NLP Seminar/Data/test_hadm_ids_top_50.csv',header=False, index=False)

## 5.4 Filter Train/Dev/Test Datasets To Top 50 ICD Codes

In [0]:
splits_top_K = {}
codes_top_K_set = set(codes_top_K.index)
for split in ['train', 'dev', 'test']:
    split_ids_top_K_df = pd.read_csv(path.join(SPLITS_DIR, '{}_hadm_ids_top_50.csv'.format(split)), names=['HADM_ID'], dtype=int32)
    splits_top_K[split] = pd.merge(dflabeled, split_ids_top_K_df, how='inner', on=['HADM_ID'])
    splits_top_K[split]['LABELS'] = splits_top_K[split]['LABELS'].apply(lambda labels: list(codes_top_K_set.intersection(set(labels))))

In [0]:
print(splits_top_K)

{'train':        SUBJECT_ID  ...                                             LABELS
0               3  ...  [38.93, 96.6, 584.9, 96.72, 038.9, 428.0, 410....
1               4  ...                                     [88.72, 38.93]
2               6  ...                             [99.04, 285.9, V15.82]
3               9  ...         [401.9, 584.9, 96.72, 507.0, 428.0, 96.04]
4              10  ...                                      [96.6, 99.15]
...           ...  ...                                                ...
34456       48812  ...  [995.92, 38.91, 38.93, 584.9, 038.9, 507.0, 31...
34457       48821  ...  [39.95, 272.4, 250.00, 414.01, 36.15, 39.61, 4...
34458       48821  ...  [272.4, 250.00, 584.9, 414.01, 428.0, V45.81, ...
34459       48826  ...  [995.92, 486, 276.2, 96.6, 038.9, 96.72, 276.1...
34460       48826  ...               [401.9, 511.9, 584.9, 507.0, V45.81]

[34461 rows x 5 columns], 'dev':        SUBJECT_ID  ...                                             L

In [0]:
train_df= splits_top_K['train']
test_df= splits_top_K['test']
dev_df= splits_top_K['dev']


In [0]:
train_df.to_csv('/content/drive/Shared drives/NLP Seminar/Data/Train_Tokenized_Labled_Top_K_Labels')
test_df.to_csv('/content/drive/Shared drives/NLP Seminar/Data/Test_Tokenized_Labled_Top_K_Labels')
dev_df.to_csv('/content/drive/Shared drives/NLP Seminar/Data/Dev_Tokenized_Labled_Top_K_Labels')

# 6.Auxillary Function
Used to Format ICD Codes

In [0]:
def reformat(code, is_diag):
    """
        Put a period in the right place because the MIMIC-3 data files exclude them.
        Generally, procedure codes have dots after the first two digits, 
        while diagnosis codes have dots after the first three digits.
    """
    if code == '':
        return code
        
    code = ''.join(code.split('.'))
    if is_diag:
        if code.startswith('E'):
            if len(code) > 4:
                code = code[:4] + '.' + code[4:] if len(code) > 4 else code
        else:
            code = code[:3] + '.' + code[3:] if len(code) > 3 else code
    else:
        code = code[:2] + '.' + code[2:] if len(code) > 2 else code
    return code

# !-------------------------------------------!
